In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"
# 코드에서 nms제외해야함

In [2]:
# !pip install onnxruntime
# !pip install onnx onnxconverter-common
# !pip install -U onnxruntime


In [3]:
import numpy as np
import tensorrt as trt
import torch
from pcdet.config import cfg, cfg_from_yaml_file
from pcdet.models import build_network
from pcdet.datasets import build_dataloader
from pcdet.utils import common_utils
# import pycuda.driver as cuda
# import pycuda.autoinit
# import torch_tensorrt

# import pycuda.driver as cuda
# import pycuda.autoinit

fatal: detected dubious ownership in repository at '/mnt/nas2/users/eslim/workspace/OpenPCDet'
To add an exception for this directory, call:

	git config --global --add safe.directory /mnt/nas2/users/eslim/workspace/OpenPCDet


In [4]:

cfg_file = "./pillar/config.yaml"
cfg_from_yaml_file(cfg_file, cfg)
if os.path.exists('./deploy_pillar_sfaw_bm')==False:
    os.mkdir('./deploy_pillar_sfaw_bm')
log_file = './deploy_pillar_sfaw_bm/log_trt.log'

logger = common_utils.create_logger(log_file, rank=0)
test_set, test_loader, sampler = build_dataloader(
    dataset_cfg=cfg.DATA_CONFIG,
    class_names=cfg.CLASS_NAMES,
    batch_size=1,
    dist=False, workers=8, logger=logger, training=False
)

model = build_network(model_cfg=cfg.MODEL, num_class=len(cfg.CLASS_NAMES), dataset=test_set)
ckpt = "./pillar/model.pth"

model.load_params_from_file(filename=ckpt, logger=logger, to_cpu=False, pre_trained_path=None)
model.eval()
model.cuda()
####### load model #######

####### read input #######
batch_dict = torch.load("/mnt/nas2/users/eslim/onnx/sfaw/input_dict.pth", map_location="cuda")
inputs = batch_dict
# inputs_vfe = model.vfe(inputs)


2024-04-16 13:26:14,949   INFO  Loading Custom dataset.
2024-04-16 13:26:14,951   INFO  Total samples for CUSTOM dataset: 0
2024-04-16 13:26:17,150   INFO  ==> Loading parameters from checkpoint ./pillar/model.pth to GPU
2024-04-16 13:26:17,268   INFO  ==> Checkpoint trained from version: pcdet+0.6.0+255db8f
2024-04-16 13:26:17,310   INFO  ==> Done (loaded 391/391)


In [5]:
def load_trt_engine(path: str) -> trt.ICudaEngine:
    """Deserialize TensorRT engine from disk.

    Args:
        path (str): The disk path to read the engine.

    Returns:
        tensorrt.ICudaEngine: The TensorRT engine loaded from disk.
    """
    # load_tensorrt_plugin()
    with trt.Logger() as logger, trt.Runtime(logger) as runtime:
        with open(path, mode='rb') as f:
            engine_bytes = f.read()
        engine = runtime.deserialize_cuda_engine(engine_bytes)
        print(f"TensorRT engine {path} successfully loaded.")
        return engine

In [6]:
def load_context(engine):

    trt.init_libnvinfer_plugins(None, '')
    if isinstance(trt_path, str):
        engine = load_trt_engine(trt_path)
    
    if not isinstance(engine, trt.ICudaEngine):
        raise TypeError(f'`engine` should be str or trt.ICudaEngine, \
            but given: {type(engine)}')
    
    context = engine.create_execution_context()
    return context

In [7]:
# context.set_tensor_address("src", ptr)
trt_path = './deploy_pillar_sfaw_3d_origin/dsvt_3d_bb_origin_2.engine'
engine = load_trt_engine(trt_path)
context = load_context(engine)

TensorRT engine ./deploy_pillar_sfaw_3d_origin/dsvt_3d_bb_origin_2.engine successfully loaded.
TensorRT engine ./deploy_pillar_sfaw_3d_origin/dsvt_3d_bb_origin_2.engine successfully loaded.
[04/16/2024-13:26:18] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage and speed up TensorRT initialization. See "Lazy Loading" section of CUDA documentation https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#lazy-loading


In [8]:
input_names = [
    'src',
    'set_voxel_inds_tensor_shift_0', 
    'set_voxel_inds_tensor_shift_1', 
    'set_voxel_masks_tensor_shift_0', 
    'set_voxel_masks_tensor_shift_1',
    'pos_embed_tensor'
]
output_names = ["output",]

DSVT_Backbone = model.backbone_3d
dsvtblocks_list = DSVT_Backbone.stage_0
layer_norms_list = DSVT_Backbone.residual_norm_stage_0

inputs_vfe = model.vfe(inputs)
import time
start_time = time.time()
inputs_3d = model.backbone_3d(inputs_vfe)
print(time.time()-start_time)
voxel_info = DSVT_Backbone.input_layer(inputs_vfe)
voxel_feat = voxel_info['voxel_feats_stage0']

pillar_features = inputs_vfe['voxel_features']
set_voxel_inds_list = [[voxel_info[f'set_voxel_inds_stage{s}_shift{i}'] for i in range(2)] for s in range(1)]
set_voxel_masks_list = [[voxel_info[f'set_voxel_mask_stage{s}_shift{i}'] for i in range(2)] for s in range(1)]
pos_embed_list = [[[voxel_info[f'pos_embed_stage{s}_block{b}_shift{i}'] for i in range(2)] for b in range(4)] for s in range(1)]

output = voxel_feat
inputs = dict(
        src=output,
        set_voxel_inds_tensor_shift_0=set_voxel_inds_list[0][0].int(),
        set_voxel_inds_tensor_shift_1=set_voxel_inds_list[0][1].int(),
        set_voxel_masks_tensor_shift_0=set_voxel_masks_list[0][0],
        set_voxel_masks_tensor_shift_1=set_voxel_masks_list[0][1],
        pos_embed_tensor=torch.stack([torch.stack(v, dim=0) for v in pos_embed_list[0]], dim=0),
    )

0.07900428771972656


In [9]:
bindings = [None] * (len(input_names) + len(output_names))

# https://github.dev/Haiyang-W/DSVT , dsvt.py -> 384line
profile_id = 0
idx = 0
for input_name, input_tensor in inputs.items():    
    profile = engine.get_profile_shape(profile_id,input_name)
    print(f'profile : {profile}')
    print(f'type : {input_tensor.dtype}')
    print(f'shape : {input_tensor.size()}')
    context.set_input_shape(input_name,tuple(input_tensor.shape))
    bindings[idx] = input_tensor.contiguous().data_ptr()
    idx+=1
# get_binding_index

profile : [(962, 192), (962, 192), (962, 192)]
type : torch.float32
shape : torch.Size([962, 192])
profile : [(2, 40, 36), (2, 40, 36), (2, 40, 36)]
type : torch.int32
shape : torch.Size([2, 40, 36])
profile : [(2, 34, 36), (2, 34, 36), (2, 34, 36)]
type : torch.int32
shape : torch.Size([2, 34, 36])
profile : [(2, 40, 36), (2, 40, 36), (2, 40, 36)]
type : torch.bool
shape : torch.Size([2, 40, 36])
profile : [(2, 34, 36), (2, 34, 36), (2, 34, 36)]
type : torch.bool
shape : torch.Size([2, 34, 36])
profile : [(4, 2, 962, 192), (4, 2, 962, 192), (4, 2, 962, 192)]
type : torch.float32
shape : torch.Size([4, 2, 962, 192])


/tmp/ipykernel_3265058/1391992233.py:7: DeprecationWarning: Use get_tensor_profile_shape instead.
  profile = engine.get_profile_shape(profile_id,input_name)


In [10]:
bindings

[140227549069312,
 140229592588288,
 140229592600064,
 140229597966848,
 140229592652800,
 140227393880064,
 None]

In [11]:
bindings[-1] = output.data_ptr()
bindings




[140227549069312,
 140229592588288,
 140229592600064,
 140229597966848,
 140229592652800,
 140227393880064,
 140227549069312]

In [12]:
import time
start_time = time.time()
context.execute_v2(bindings)
print(time.time()-start_time)

0.00435328483581543


In [13]:
output

tensor([[-0.7080,  0.1649,  0.0999,  ..., -1.1533,  0.0462, -0.0028],
        [-0.7070,  0.1465,  0.1014,  ..., -1.1719,  0.0542,  0.0415],
        [-0.8643,  0.3259,  0.1472,  ..., -1.1963,  0.1998,  0.0409],
        ...,
        [-0.3025,  0.1978,  0.0900,  ..., -1.1904,  0.0031,  0.1912],
        [-0.2747,  0.3757,  0.1881,  ..., -1.3145, -0.2715,  0.0710],
        [-0.2681,  0.2710,  0.2347,  ..., -1.1846,  0.1216, -0.0175]],
       device='cuda:0', grad_fn=<CloneBackward0>)

In [14]:
# context.set_tensor_address("src", ptr)
trt_path = './deploy_pillar_sfaw_2d/dsvt_2d_bb_2.engine'
engine = load_trt_engine(trt_path)
context = load_context(engine)


TensorRT engine ./deploy_pillar_sfaw_2d/dsvt_2d_bb_2.engine successfully loaded.
TensorRT engine ./deploy_pillar_sfaw_2d/dsvt_2d_bb_2.engine successfully loaded.
[04/16/2024-13:26:19] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage and speed up TensorRT initialization. See "Lazy Loading" section of CUDA documentation https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#lazy-loading


In [15]:
inputs_3d = model.backbone_3d(inputs_vfe)
input_names = [
    # 'voxel_features',
    # 'use_lead_xyz',
    # 'batch_size',
    'pillar_features',
    'voxel_coords',
]
output_names = ["output"]


input_dict = {}
input_dict['pillar_features'] = inputs_3d['pillar_features']
input_dict['voxel_coords'] = inputs_3d['voxel_coords']
output = torch.empty([1, 192, 468, 468]).to('cuda:0')



In [16]:
bindings = [None] * (len(input_names) + len(output_names))

# https://github.dev/Haiyang-W/DSVT , dsvt.py -> 384line
profile_id = 0
idx = 0
for input_name, input_tensor in input_dict.items():    
    profile = engine.get_profile_shape(profile_id,input_name)
    if input_tensor.dtype == torch.long:
        input_tensor=input_tensor.int()
        print("convert long to int")
    print(f'input_name: {input_name}')
    print(f'profile : {profile}')
    print(f'type : {input_tensor.dtype}')
    print(f'shape : {input_tensor.size()}')
    print("======")
    context.set_input_shape(input_name,tuple(input_tensor.shape))
    bindings[idx] = input_tensor.contiguous().data_ptr()
    idx+=1
# get_binding_index
bindings[-1] = output.data_ptr()


input_name: pillar_features
profile : [(962, 192), (962, 192), (962, 192)]
type : torch.float32
shape : torch.Size([962, 192])
convert long to int
input_name: voxel_coords
profile : [(962, 4), (962, 4), (962, 4)]
type : torch.int32
shape : torch.Size([962, 4])


/tmp/ipykernel_3265058/854044026.py:7: DeprecationWarning: Use get_tensor_profile_shape instead.
  profile = engine.get_profile_shape(profile_id,input_name)


In [17]:
bindings

[140227289761280, 140227367413760, 140226588573696]

In [18]:
start_time = time.time()
context.execute_v2(
            bindings
        )
print(time.time()-start_time)

0.025560379028320312[04/16/2024-13:26:19] [TRT] [E] 1: [reformat.cpp::executeCutensor::331] Error Code 1: CuTensor (Internal cuTensor permutate execute failed)

